In [2]:
import ee 
import geemap
import pandas as pd
from ltgee import LandTrendr, LandsatComposite, LtCollection
from datetime import date

In [3]:
ee.Initialize()

In [5]:
# read in fire-insect co-occurrence data
occurrence = pd.read_csv('/home/goldma34/fire_insect_co-occurence/data/outputs/on/on_co-occurrences_1986-2012.csv')

# get fire names and turn it into a list
fire_names = occurrence['Fire_ID'].unique().tolist()

# Read in fires from GEE
# Filter the fires by the names in the list
fires = ee.FeatureCollection("users/jandrewgoldman/Ont_BurnSeverity_Trends/ON_FirePerimeters_85to2020_v0").filter(ee.Filter.inList('Fire_ID', fire_names))


In [61]:
feature = fires.first()

feature.getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-70.5624450551198, 49.948571782368454],
    [-70.54948626426713, 49.94519000402315],
    [-70.53374607733822, 49.970283615347896],
    [-70.5467114582775, 49.973667739865725],
    [-70.5624450551198, 49.948571782368454]]]},
 'id': '00000000000000000038',
 'properties': {'ADJ_FLAG': 1,
  'ADJ_HA': 317.77410967,
  'AFEDATE': None,
  'AFSDATE': None,
  'BASRC': 2,
  'BT_GID': 79,
  'BURNCLAS': 4,
  'CAPDATE': None,
  'COMMENTS': '',
  'EDATE': 1063868400000,
  'FIRECAUS': 0,
  'FIREMAPM': 6,
  'FIREMAPS': 7,
  'Fire_ID': 'QC_655_2003',
  'Fire_Year': '2003',
  'POLY_HA': 302.176069771,
  'SDATE': 1063263600000,
  'VERSION': '2003_r9_20210810',
  'nfireid': '655',
  'prov': 'QC'}}

In [62]:
fire_id = feature.get('Fire_ID').getInfo()
geometry = feature.geometry().bounds()
fire_year = feature.get('Fire_Year').getInfo()

 # turn fire year into a string and get prefire year
fire_year = str(fire_year)

print(fire_year)

2003


In [ ]:
prefire_year = str(int(fire_year) - 1)
postfire_year = str(int(fire_year) + 11)
print(prefire_year)


2002


In [19]:
# Example arguments to pass to the LandTrendr class. See docstring of LandTrendr for more information'
composite_params = {
    "start_date": date(int(prefire_year), 4,1),
    "end_date": date(int(postfire_year), 11,1),
    "area_of_interest": geometry,
    "mask_labels": ['cloud', 'shadow', 'snow', 'water'],
    "debug": True
}
lt_collection_params = {
        "sr_collection": LandsatComposite(**composite_params),
        # "sr_collection": composite_params, # - you may also just pass in your own collection or the params directly
        "index": 'NBR',
        "ftv_list": ['NBR'],
}
lt_params = {
    "lt_collection": LtCollection(**lt_collection_params),
    # "lt_collection": lt_collection_params, # - you may also just pass in your own collection or the params directly
    "run_params": {
            "maxSegments": 6,
            "spikeThreshold": 0.9,
            "vertexCountOvershoot":  3,
            "preventOneYearRecovery":  True,
            "recoveryThreshold":  0.25,
            "pvalThreshold":  .05,
            "bestModelProportion":  0.75,
            "minObservationsNeeded": 2,
        }
}

In [20]:
lt = LandTrendr(**lt_params)
lt

In [28]:
ftv_nbr = lt.select('ftv_nbr_fit').clip(composite_params['area_of_interest'])
ftv_nbr

In [18]:
# Initialize an empty list to store the year strings
years = []

# Iterate over the range of years and append the formatted strings to the list
for i in range(int(prefire_year), int(postfire_year) + 1):
    years.append('yr' + str(i))

NameError: name 'prefire_year' is not defined

In [43]:
nbr_stack = ftv_nbr.arrayFlatten([years])
nbr_stack                  

In [6]:
def rename_bands(image):
    """
    Renames each band in the image by stripping "yr" from the name.

    Parameters:
    image (ee.Image): The input image with bands to be renamed.

    Returns:
    ee.Image: The image with renamed bands.
    """
    # Get the list of band names
    band_names = image.bandNames()

    # Create a list of new band names by stripping "yr" from each name
    new_band_names = band_names.map(lambda name: ee.String(name).replace('yr', '', 'g'))

    # Rename the bands in the image
    renamed_image = image.rename(new_band_names)

    return renamed_image

    

In [16]:
new_names = rename_bands(nbr_stack)
new_names

NameError: name 'rename_bands' is not defined

In [7]:
def calcBS(img, ft1):
    ft1 = ee.Feature(ft1)
    
    # Select the bands by their indices
    preNBR = img.select([0]).rename('preNBR')
    postNBR = img.select([2]).rename('postNBR')
    
    # Calculate dNBR
    dnbr = preNBR.subtract(postNBR).multiply(1000).rename('dnbr').toFloat()
    
    # Create a ring buffer around the feature
    ring = ft1.buffer(180).difference(ft1)
    
    # Calculate the offset
    offset = ee.Image.constant(ee.Number(dnbr.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ring.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('dnbr'))).rename('offset').toFloat()
    
    # Add the offset and original bands to the dNBR image
    dnbr = dnbr.addBands(offset).addBands(preNBR).addBands(postNBR)
    
    # Calculate dNBR with offset
    dnbr_offset = dnbr.expression("b('dnbr') - b('offset')").rename('dnbr_w_offset').toFloat()
    dnbr_offset = dnbr_offset.addBands(dnbr)
    
    # Calculate RBR
    rbr = dnbr_offset.expression("b('dnbr') / (b('preNBR') + 1.001)").rename('rbr').toFloat().addBands(dnbr_offset)
    
    # Calculate RBR with offset
    rbr_offset = rbr.expression("b('dnbr_w_offset') / (b('preNBR') + 1.001)").rename('rbr_w_offset').toFloat().addBands(rbr)
    
    # Set properties
    rbr_offset = rbr_offset.set('fireID', ft1.get('Fire_ID'), 'fireName', ft1.get('Fire_Name'), 'fireYear', ft1.get('Fire_Year'))
    
    converted_to_float = rbr_offset.toFloat()

    return converted_to_float



In [119]:
results = calcBS(new_names, feature)
results

In [117]:
results_float = results.toFloat()
print('Original band types:', results.bandTypes().getInfo())
print('Converted band types:', results_float.bandTypes().getInfo())

Original band types: {'dnbr': {'type': 'PixelType', 'precision': 'float'}, 'dnbr_w_offset': {'type': 'PixelType', 'precision': 'float'}, 'offset': {'type': 'PixelType', 'precision': 'float'}, 'postNBR': {'type': 'PixelType', 'precision': 'double'}, 'preNBR': {'type': 'PixelType', 'precision': 'double'}, 'rbr': {'type': 'PixelType', 'precision': 'float'}, 'rbr_w_offset': {'type': 'PixelType', 'precision': 'float'}}
Converted band types: {'dnbr': {'type': 'PixelType', 'precision': 'float'}, 'dnbr_w_offset': {'type': 'PixelType', 'precision': 'float'}, 'offset': {'type': 'PixelType', 'precision': 'float'}, 'postNBR': {'type': 'PixelType', 'precision': 'float'}, 'preNBR': {'type': 'PixelType', 'precision': 'float'}, 'rbr': {'type': 'PixelType', 'precision': 'float'}, 'rbr_w_offset': {'type': 'PixelType', 'precision': 'float'}}


In [10]:
def export_landtrendr_image(feature):

    # get the feature's ID, geometry and year
    fire_id = feature.get('Fire_ID').getInfo()
    geometry = feature.geometry().bounds()
    fire_year = feature.get('Fire_Year').getInfo()
    print(f"fire check {fire_year}")

    if fire_id is None or fire_year is None:
        print(f"Skipping fire with missing Fire_ID or Fire_Year: {fire_id}, {fire_year}")
        return

    # turn fire year into a string and get prefire year
    fire_year = str(fire_year)
    prefire_year = str(int(fire_year) - 1)
    postfire_year = str(int(fire_year) + 11)

    # Example arguments to pass to the LandTrendr class. See docstring of LandTrendr for more information'
    composite_params = {
    "start_date": date(int(prefire_year), 4,1),
    "end_date": date(int(postfire_year), 11,1),
    "area_of_interest": geometry,
    "mask_labels": ['cloud', 'shadow', 'snow', 'water'],
    "debug": True
}
    lt_collection_params = {
        "sr_collection": LandsatComposite(**composite_params),
        # "sr_collection": composite_params, # - you may also just pass in your own collection or the params directly
        "index": 'NBR',
        "ftv_list": ['NBR']}
    lt_params = {
        "lt_collection": LtCollection(**lt_collection_params),
        # "lt_collection": lt_collection_params, # - you may also just pass in your own collection or the params directly
        "run_params": {
                "maxSegments": 6,
                "spikeThreshold": 0.9,
                "vertexCountOvershoot":  3,
                "preventOneYearRecovery":  True,
                "recoveryThreshold":  0.25,
                "pvalThreshold":  .05,
                "bestModelProportion":  0.75,
                "minObservationsNeeded": 2,
            }
    }

    lt = LandTrendr(**lt_params)
    lt.run()
    
    ftv_nbr = lt.select('ftv_nbr_fit').clip(composite_params['area_of_interest'])

    years = []

    # Iterate over the range of years and append the formatted strings to the list
    for i in range(int(prefire_year), int(postfire_year) + 1):
        years.append('yr' + str(i))


    # convert 1-D fitted nbr array to an image with bands representing years
    nbr_stack = ftv_nbr.arrayFlatten([years])

    # clean the names of the nbr stack bands
    clean_nbr_stack = rename_bands(nbr_stack)

    # calculate burn severity indices
    results = calcBS(clean_nbr_stack, feature)

    # make the file names to export each image
    export_bi_name = fire_id + "_bi"
    export_nbr_name = fire_id + "_nbr"
    
    # export  burn Indices imagery
    geemap.ee_export_image_to_drive(results, description=export_bi_name , folder="chapter3_lt_bi_v2_on", region=geometry, scale=30)

    # export nbr recovery imagery
    geemap.ee_export_image_to_drive(clean_nbr_stack, description=export_nbr_name, folder="chapter3_lt_nbr_v2_on", region=geometry, scale=30)
    


    



In [9]:
fire_ids = fires.aggregate_array('Fire_ID').getInfo()

In [11]:

for fire_id in fire_ids:
        print(fire_id)
        fire = fires.filterMetadata('Fire_ID', 'equals', fire_id).first()
        results = export_landtrendr_image(fire)
        print(f"Exported LandTrendr imagery for fire {fire_id}")


FOR14_2005_1514
fire check 2005
Exported LandTrendr imagery for fire FOR14_2005_1514
FOR24_2005_1516
fire check 2005
Exported LandTrendr imagery for fire FOR24_2005_1516
FOR18_2005_1520
fire check 2005
Exported LandTrendr imagery for fire FOR18_2005_1520
SUD1_2005_1524
fire check 2005
Exported LandTrendr imagery for fire SUD1_2005_1524
TER7_1986_83
fire check 1986
Exported LandTrendr imagery for fire TER7_1986_83
COC11_1986_1332
fire check 1986
Exported LandTrendr imagery for fire COC11_1986_1332
KAP10_1986_1340
fire check 1986
Exported LandTrendr imagery for fire KAP10_1986_1340
CHA9_1987_133
fire check 1987
Exported LandTrendr imagery for fire CHA9_1987_133
GOG1_1987_468
fire check 1987
Exported LandTrendr imagery for fire GOG1_1987_468
HEA6_1987_479
fire check 1987
Exported LandTrendr imagery for fire HEA6_1987_479
IGN9_1987_492
fire check 1987
Exported LandTrendr imagery for fire IGN9_1987_492
FOR33_1987_639
fire check 1987
Exported LandTrendr imagery for fire FOR33_1987_639
THU71_

In [13]:
import os

# Define the folder path and the list of names to check against
folder_path = '/home/goldma34/fire_insect_co-occurence/data/chapter3_lt_nbr_v2/'  # Replace with your actual folder path
names_list = fire_ids  # Replace with your actual list of names

# Read all file names from the folder
file_names = os.listdir(folder_path)

# Clean the file names by removing the "_nbr.tif" suffix
cleaned_names = [name.replace('_nbr.tif', '') for name in file_names]

# Filter out the cleaned names from the names_list
filtered_names_list = [name for name in fire_ids if name not in cleaned_names]

# Print the results
print("Cleaned Names:", cleaned_names)
print("Filtered Names List:", filtered_names_list)

Cleaned Names: ['QC_677_2012', 'QC_1261_2009', 'QC_1079_1996', 'QC_752_2002', 'QC_731_1996', 'QC_777_2010', 'QC_385_2009', 'QC_1461_1995', 'QC_350_2007', 'QC_794_1996', 'QC_95_1991', 'QC_772_2010', 'QC_1251_2009', 'QC_768_1996', 'QC_397_2001', 'QC_719_1999', 'QC_770_2010', 'QC_439_2002', 'QC_733_1996', 'QC_551_2000', 'QC_1166_1995', 'QC_725_1999', 'QC_1449_1995', 'QC_304_1997', 'QC_35_2007', 'QC_632_2011', 'QC_377_2009', 'QC_993_2000', 'QC_1436_1995', 'QC_1474_1995', 'QC_744_1996', 'QC_546_2000', 'QC_951_2001', 'QC_335_2007', 'QC_428_2011', 'QC_907_2003', 'QC_443_1997', 'QC_641_2011', 'QC_1522_1998', 'QC_1262_2009', 'QC_1159_1995', 'QC_448_1997', 'QC_750_2002', 'QC_447_1997', 'QC_457_2009', 'QC_1456_1995', 'QC_333_2007', 'QC_475_2005', 'QC_735_1996', 'QC_739_1996', 'QC_1076_1996', 'QC_728_1999', 'QC_917_2012', 'QC_1168_1999', 'QC_741_1996', 'QC_292_2007', 'QC_324_2007', 'QC_902_1998', 'QC_1435_1995', 'QC_1122_1996', 'QC_3_1994', 'QC_740_1996', 'QC_467_1996', 'QC_297_1997', 'QC_50_1988'

In [24]:
for fire_id in filtered_names_list:
        print(fire_id)
        fire = fires.filterMetadata('Fire_ID', 'equals', fire_id).first()
        results = export_landtrendr_image(fire)
        print(f"Exported LandTrendr imagery for fire {fire_id}")

QC_1783_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1783_2010
QC_1812_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1812_2010
QC_1824_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1824_2010
QC_1806_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1806_2010
QC_1816_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1816_2010
QC_1810_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1810_2010
QC_1845_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1845_2010
QC_1798_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1798_2010
QC_1825_2010
fire check 2010
Exported LandTrendr imagery for fire QC_1825_2010
QC_1201_2011
fire check 2011
Exported LandTrendr imagery for fire QC_1201_2011
QC_547_2011
fire check 2011
Exported LandTrendr imagery for fire QC_547_2011
QC_1192_2011
fire check 2011
Exported LandTrendr imagery for fire QC_1192_2011
QC_1188_2011
fire check 2011
Exported LandTrendr image